In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork as nn
from tfomics import neuralbuild as nb
from tfomics import utils, fit, explore

In [2]:
def load_simulation(filepath):
    # setup paths for file handling

    trainmat = h5py.File(filepath, 'r')

    print("loading training data")
    X_train = np.array(trainmat['X_train']).astype(np.float32)
    y_train = np.array(trainmat['Y_train']).astype(np.float32)

    print("loading cross-validation data")
    X_valid = np.array(trainmat['X_valid']).astype(np.float32)
    y_valid = np.array(trainmat['Y_valid']).astype(np.int32)

    print("loading test data")
    X_test = np.array(trainmat['X_test']).astype(np.float32)
    y_test = np.array(trainmat['Y_test']).astype(np.int32)


    X_train = np.expand_dims(X_train, axis=3).transpose([0,2,3,1])
    X_valid = np.expand_dims(X_valid, axis=3).transpose([0,2,3,1])
    X_test = np.expand_dims(X_test, axis=3).transpose([0,2,3,1])

    return X_train, y_train, X_valid, y_valid, X_test, y_test


# load data
filename = 'Synthetic_TF_dataset_20000.h5'
data_path = '/Users/juliankimura/Desktop/representation learning/data/synthetic_TF_dataset'
filepath = os.path.join(data_path, filename)
X_train, y_train, X_valid, y_valid, X_test, y_test = load_simulation(filepath)

loading training data
loading cross-validation data
loading test data


In [3]:

def model(input_shape, num_labels):


    # create model
    layer1 = {  'layer': 'input',
                'input_shape': input_shape,
                }
    layer2 = {  'layer': 'conv1d', 
                'num_filters': {'start': 32, 'bounds': [1, 200], 'scale': 25, 'multiples': 4},
                'filter_size': {'start': 7, 'bounds': [5, 32], 'scale': 5, 'odd': True, 'multiples': 2},
                'norm': 'batch',
                'padding': 'SAME',
                'activation': 'relu',
                'dropout': {'start': 0.2, 'bounds': [0., 0.6], 'scale': 0.1},
                'name': 'conv1'
                }
    layer3 = {  'layer': 'conv1d', 
                'num_filters': {'start': 64, 'bounds': [1, 200], 'scale': 25, 'multiples': 4},
                'filter_size': {'start': 7, 'bounds': [5, 32], 'scale': 5, 'odd': True, 'multiples': 2},
                'norm': 'batch',
                'padding': 'SAME',
                'activation': 'relu',
                'dropout': {'start': 0.2, 'bounds': [0., 0.6], 'scale': 0.1},
                'name': 'conv2'
                }
    layer4 = {  'layer': 'dense', 
                'num_units': {'start': 256, 'bounds': [16, 1000], 'scale': 50, 'multiples': 4},
                #'norm': 'batch',
                'activation': 'relu',
                'dropout': {'start': 0.4, 'bounds': [0., 0.8], 'scale': 0.2},
                'name': 'dense1'
                }            
    layer5 = {  'layer': 'dense', 
                'num_units': num_labels,
                'activation': 'softmax',
                }

    #from tfomics import build_network
    model_layers = [layer1, layer2, layer3, layer4, layer5]

    # optimization parameters
    optimization = {"objective": "categorical",
                    "optimizer": "adam",
                    "learning_rate": 0.001,
                    "l2": 1e-6
                    #"learning_rate": {'start': -3, 'bounds': [-4, -1], 'scale': 1.5, 'transform': 'log'},      
                    #"l2": {'start': -6, 'bounds': [-8, -2], 'scale': 3, 'transform': 'log'},
                    # "l1": 0, 
                    }
    return model_layers, optimization

# build network
input_shape = list(X_train.shape)
input_shape[0] = None
num_labels = y_train.shape[1]
model_layers, optimization = model(input_shape, num_labels)

# build neural opti
optimizer = explore.NeuralOptimizer(model_layers, optimization)


In [4]:
optimizer.sample_network()

[{'input_shape': [None, 200, 1, 4], 'layer': 'input'},
 {'activation': 'relu',
  'dropout': 0.2303066842769604,
  'filter_size': 9,
  'layer': 'conv1d',
  'name': 'conv1',
  'norm': 'batch',
  'num_filters': 40,
  'padding': 'SAME'},
 {'activation': 'relu',
  'dropout': 0.1958008122739223,
  'filter_size': 9,
  'layer': 'conv1d',
  'name': 'conv2',
  'norm': 'batch',
  'num_filters': 76,
  'padding': 'SAME'},
 {'activation': 'relu',
  'dropout': 0.2902731420912835,
  'layer': 'dense',
  'name': 'dense1',
  'num_units': 164},
 {'activation': 'softmax', 'layer': 'dense', 'num_units': 1}]

In [ ]:
# optimize model parameters
train = {'inputs': X_train, 'targets': y_train}
valid = {'inputs': X_valid, 'targets': y_valid}        
optimizer.optimize(train, valid, num_trials=100, num_epochs=20, batch_size=128, verbose=0)

---------------------------------------------------------
Running baseline model

Model layers:
num_filters: 32
name: conv1
filter_size: 7
dropout: 0.2
num_filters: 64
name: conv2
filter_size: 7
dropout: 0.2
dropout: 0.4
num_units: 256
name: dense1
num_units: 1

Optimization:
learning_rate: 0.001
l2: 1e-06

Epoch 1 out of 20 


In [ ]:
# save optimal model
optimizer.print_optimal_model()